In [1]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)


from pathlib import Path
import os


from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score,mean_squared_error, matthews_corrcoef
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score



#transformers and pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv





import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
from optuna.samplers import TPESampler
import warnings


matplotlib.rc('image', cmap='Greys')

from fastkaggle import setup_comp



from openfe import OpenFE, transform
from autogluon.tabular import TabularDataset, TabularPredictor

import h2o
from h2o.automl import H2OAutoML

import gc

from xgboost import plot_importance
import wandb
from fastai.callback.wandb import *
from wandb.integration.lightgbm import wandb_callback, log_summary
#from wandb.xgboost import wandb_callback



#from IPython.display import FileLink

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
path = Path('playground-series-s4e8/')
path

Path('playground-series-s4e8')

In [3]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')

In [4]:
%%time
predictor = TabularPredictor(label='class').fit(train_data=train_df)

No path specified. Models will be saved in: "AutogluonModels/ag-20240808_073211"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #70~20.04.1-Ubuntu SMP Fri Jun 14 15:42:13 UTC 2024
CPU Count:          4
Memory Avail:       11.18 GB / 15.34 GB (72.9%)
Disk Space Avail:   354.69 GB / 387.48 GB (91.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

[1000]	valid_set's binary_error: 0.00712223


	0.9931	 = Validation score   (accuracy)
	186.72s	 = Training   runtime
	1.83s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.00731473


	0.9928	 = Validation score   (accuracy)
	168.51s	 = Training   runtime
	1.42s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9926	 = Validation score   (accuracy)
	578.92s	 = Training   runtime
	0.51s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9927	 = Validation score   (accuracy)
	601.91s	 = Training   runtime
	0.62s	 = Validation runtime
Fitting model: CatBoost ...
	0.9921	 = Validation score   (accuracy)
	6951.02s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9926	 = Validation score   (accuracy)
	348.93s	 = Training   runtime
	0.5s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9925	 = Validation score   (accuracy)
	360.67s	 = Training   runtime
	0.56s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.9931	 = Validation score   (accuracy)
	1337.49s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: XGBoost ...
	0.9927	 = Validation score   (accuracy)
	558.15s	 = Training  

CPU times: user 8h 22min 11s, sys: 1min 32s, total: 8h 23min 44s
Wall time: 3h 45min 51s


In [5]:
%%time
predictions = predictor.predict(test_df)

CPU times: user 5min 36s, sys: 1.29 s, total: 5min 37s
Wall time: 2min 44s


In [4]:
predictor = TabularPredictor.load("AutogluonModels/ag-20240808_073211")

In [8]:
predictions.shape

(2077964,)

In [6]:
%%time
predictor.evaluate(train_df)

CPU times: user 8min 32s, sys: 2.6 s, total: 8min 35s
Wall time: 4min 11s


{'accuracy': 0.9928920786218557,
 'balanced_accuracy': 0.992935118361002,
 'mcc': 0.9856622699716666,
 'roc_auc': 0.9980605040169049,
 'f1': 0.9934977708110222,
 'precision': 0.9945190572637475,
 'recall': 0.9924785797550832}

In [ ]:
#predictor.evaluate(test_df)

In [27]:
%%time
predictor.leaderboard(train_df)

CPU times: user 35min 47s, sys: 23.7 s, total: 36min 11s
Wall time: 14min 20s


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.994188,0.992557,accuracy,58.379776,0.512971,578.918851,58.379776,0.512971,578.918851,1,True,5
1,RandomForestEntr,0.993989,0.992749,accuracy,64.290634,0.618115,601.906755,64.290634,0.618115,601.906755,1,True,6
2,ExtraTreesGini,0.993355,0.992557,accuracy,56.392299,0.503843,348.934303,56.392299,0.503843,348.934303,1,True,8
3,ExtraTreesEntr,0.993247,0.992461,accuracy,66.154191,0.560885,360.672849,66.154191,0.560885,360.672849,1,True,9
4,WeightedEnsemble_L2,0.992892,0.993295,accuracy,248.415682,2.372608,3826.015384,0.059310,0.001379,0.284255,2,True,14
5,NeuralNetTorch,0.992867,0.992910,accuracy,14.157990,0.149394,2298.068064,14.157990,0.149394,2298.068064,1,True,12
6,NeuralNetFastAI,0.992667,0.993070,accuracy,18.197442,0.195503,1337.493329,18.197442,0.195503,1337.493329,1,True,10
7,LightGBMXT,0.992549,0.993134,accuracy,197.299109,1.831553,186.723330,197.299109,1.831553,186.723330,1,True,3
8,LightGBM,0.992529,0.992814,accuracy,151.317322,1.421086,168.513427,151.317322,1.421086,168.513427,1,True,4
9,XGBoost,0.992504,0.992653,accuracy,71.734637,0.676967,558.146852,71.734637,0.676967,558.146852,1,True,11


In [9]:
predictions

id
3116945    e
3116946    p
3116947    p
3116948    p
3116949    e
          ..
5194904    p
5194905    p
5194906    p
5194907    e
5194908    e
Name: class, Length: 2077964, dtype: object

In [14]:
!rm submission.csv
!ls


AutogluonModels  playground-series-s4e8		     wandb
lgbm_model.pkl	 playground-series-s4e8.zip	     xgb_model.json
main.py		 poisonous_mushrooms_classification
models		 requirements.txt


In [15]:
#submit = pd.read_csv(path/'sample_submission.csv')
predictions = predictions.reset_index(drop=False)
submit = pd.DataFrame({'id': predictions['id'], 'class': predictions['class']})
submit.set_index('id', inplace=True)
submit.to_csv('submission.csv', index=True)
sub = pd.read_csv('submission.csv', index_col='id')
sub

,class
id,
3116945,e
3116946,p
3116947,p
3116948,p
3116949,e
...,...
5194904,p
5194905,p
5194906,p


In [12]:
submit = pd.read_csv(path/'sample_submission.csv')
predictions = predictions.reset_index(drop=False)
submit['class'] = predictions['class']
submit.to_csv('submission.csv', index=True)

In [13]:
sub = pd.read_csv('submission.csv')
sub

,Unnamed: 0,id,class
0,0,3116945,e
1,1,3116946,p
2,2,3116947,p
3,3,3116948,p
4,4,3116949,e
...,...,...,...
2077959,2077959,5194904,p
2077960,2077960,5194905,p
2077961,2077961,5194906,p
2077962,2077962,5194907,e


In [16]:
!kaggle competitions submit -c playground-series-s4e8 -f submission.csv -m "[AUTOGLUON] Silver Rubanza, AUTOGLUON BASELINE  "

100%|██████████████████████████████████████| 19.8M/19.8M [00:00<00:00, 46.3MB/s]
Successfully submitted to Binary Prediction of Poisonous Mushrooms

In [16]:
submit = pd.read_csv(path/'sample_submission.csv')
df = pd.DataFrame(predictions,columns=['class'])
df.head()
#submit['class']=df['class']
#submit.head()
#sol.to_csv('./[AutoML Grand Prix] Team GD Autogluon,mcc.csv',index=False)

,class
id,
3116945,e
3116946,p
3116947,p
3116948,p
3116949,e


In [17]:
submit['class']=df['class']
submit.head()

,id,class
0,3116945,NaN
1,3116946,NaN
2,3116947,NaN
3,3116948,NaN
4,3116949,NaN


In [ ]:

sol.to_csv('./[AutoML Grand Prix] Team GD Autogluon,mcc.csv',index=False)

In [13]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['class'] = predictions
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

,id,class
0,3116945,NaN
1,3116946,NaN
2,3116947,NaN
3,3116948,NaN
4,3116949,NaN
...,...,...
2077959,5194904,NaN
2077960,5194905,NaN
2077961,5194906,NaN
2077962,5194907,NaN


In [ ]:
mapping = dict(enumerate(dls.vocab))
submit = pd.read_csv(path/'sample_submission.csv')
submit['class'] = [mapping[pred.item()] for pred in rounded_xgb_lgbm_nn_sub_preds]
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
print(sub)

In [ ]:
# Make a copy of the training dataframe to avoid modifying the original
train_df_with_nan = train_df.copy()

# Add a NaN value to a random row in the 'stem-height' column
random_index = np.random.choice(train_df_with_nan.index)
train_df_with_nan.loc[random_index, 'stem-height'] = np.nan

# Verify the NaN was added
print(train_df_with_nan['stem-height'].isna().sum())